In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


학습 파이프라인 이동하기

In [1]:
!git clone https://github.com/WheelWear/VTON-project.git

Cloning into 'VTON-project'...
remote: Enumerating objects: 358, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 358 (delta 21), reused 27 (delta 10), pack-reused 316 (from 2)
Receiving objects: 100% (358/358), 184.32 MiB | 14.83 MiB/s, done.
Resolving deltas: 100% (146/146), done.


In [2]:
%cd VTON-project/

/content/VTON-project


In [3]:
!git clone https://github.com/Zheng-Chong/CatVTON.git

Cloning into 'CatVTON'...
remote: Enumerating objects: 1345, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 1345 (delta 50), reused 43 (delta 32), pack-reused 1273 (from 2)
Receiving objects: 100% (1345/1345), 16.74 MiB | 14.41 MiB/s, done.
Resolving deltas: 100% (447/447), done.


In [4]:
!cp /content/VTON-project/pipeline_train.py /content/VTON-project/CatVTON/model/

In [5]:
ls

app.py            metrics/           requirements.txt     sam2/
CatVTON/          notebook/          resource/            toy_experiments/
dataset_v1.4.zip  pipeline_train.py  run_hyper_train.ps1  train_lora_colab.py
making_datasets/  README.md          run_hyper_train.sh   train_lora.py


In [6]:
!unzip dataset_v1.4.zip

Archive:  dataset_v1.4.zip
   creating: dataset/
   creating: dataset/cloth/
   creating: dataset/cloth/lower_img/
  inflating: dataset/cloth/lower_img/00000.jpg  
  inflating: dataset/cloth/lower_img/00001.jpg  
  inflating: dataset/cloth/lower_img/00002.jpg  
  inflating: dataset/cloth/lower_img/00003.jpg  
  inflating: dataset/cloth/lower_img/00004.jpg  
  inflating: dataset/cloth/lower_img/00005.jpg  
  inflating: dataset/cloth/lower_img/00006.jpg  
  inflating: dataset/cloth/lower_img/00007.jpg  
  inflating: dataset/cloth/lower_img/00008.jpg  
  inflating: dataset/cloth/lower_img/00009.jpg  
  inflating: dataset/cloth/lower_img/00010.jpg  
  inflating: dataset/cloth/lower_img/00011.jpg  
  inflating: dataset/cloth/lower_img/00012.jpg  
  inflating: dataset/cloth/lower_img/00013.jpg  
  inflating: dataset/cloth/lower_img/00014.jpg  
  inflating: dataset/cloth/lower_img/00015.jpg  
  inflating: dataset/cloth/lower_img/00016.jpg  
  inflating: dataset/cloth/lower_img/00017.jpg  
  i

In [7]:
pip install -r requirements.txt

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-bqqylvwp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-bqqylvwp
  Resolved https://github.com/huggingface/diffusers.git to commit a4c1aac3ae10172f4acb8eaf83aac7f1f6e02ab0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB

In [ ]:
import os
import sys
import torch
from huggingface_hub import snapshot_download, hf_hub_download
from tqdm import tqdm
from diffusers import DDPMScheduler
from peft import get_peft_model, LoraConfig, TaskType, get_peft_model_state_dict
from PIL import Image
import numpy as np
import cv2
import re  # 파일 이름 파싱용
import datetime
# 프로젝트 루트 설정 (CatVTON 모듈 접근)
project_root_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "CatVTON"))
sys.path.append(project_root_dir)
from model.cloth_masker import AutoMasker
from model.pipeline import CatVTONPipeline
from utils import compute_vae_encodings, tensor_to_image, numpy_to_pil, prepare_image, prepare_mask_image, init_weight_dtype

# 데이터셋 및 출력 경로 설정
output_dir = "infer"  # 결과 및 LoRA 가중치 저장 디렉토리
os.makedirs(output_dir, exist_ok=True)

# GPU 사용 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# CatVTON 모델 다운로드
repo_path = snapshot_download(repo_id="zhengchong/CatVTON")
print(f"Model downloaded to: {repo_path}")

# AutoMasker 초기화
automasker = AutoMasker(
    densepose_ckpt=os.path.join(repo_path, "DensePose"),
    schp_ckpt=os.path.join(repo_path, "SCHP"),
    device=device,
)

# 파이프라인 초기화
pipeline = CatVTONPipeline(
    base_ckpt="booksforcharlie/stable-diffusion-inpainting",
    attn_ckpt="zhengchong/CatVTON",
    attn_ckpt_version="mix",
    device=device,
    skip_safety_check=True,
    weight_dtype=init_weight_dtype("fp16")
)


# Hugging Face에서 LoRA 가중치 다운로드
repo_id = "Coldbrew9/wheel-CatVTON"
lora_filename = "first_best_loss_lora_r4_model_14.pt"  # 예시 파일명, 실제 파일명으로 수정
lora_weights_path = hf_hub_download(
    repo_id=repo_id,
    filename=lora_filename,
    local_dir=output_dir,
    repo_type="model"
)
print(f"Downloaded LoRA weights from {repo_id} to {lora_weights_path}")

# 파일 이름에서 lora_rank 추출
filename_base = os.path.basename(lora_weights_path)
match = re.search(r"lora_r(\d+)_model", filename_base)
if match:
    lora_rank = int(match.group(1))  # "r" 뒤의 숫자 추출
    print(f"Extracted lora_rank from filename: {lora_rank}")
else:
    raise ValueError(f"Could not extract lora_rank from filename: {filename_base}")

# LoRA 설정 (lora_rank를 파일 이름에서 가져옴)
lora_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_rank * 2,  # 학습 시와 동일한 비율 유지
    lora_dropout=0.1,
    target_modules=["to_q", "to_k", "to_v"],
)
model = get_peft_model(pipeline.unet, lora_config)
model = model.to(device)
pipeline.unet = model

# 학습된 LoRA 가중치 로드
if os.path.exists(lora_weights_path):
    lora_state_dict = torch.load(lora_weights_path, map_location=device)
    model.load_state_dict(lora_state_dict, strict=False)
    print(f"Loaded LoRA weights from {lora_weights_path}")
else:
    raise FileNotFoundError(f"LoRA weights not found at {lora_weights_path}")

# 데이터 준비
data_root_path = "C:/Users/coldbrew/VTON-project/dataset"
person_item = "00003.jpg"  # 사람 이미지 파일명
person_path = os.path.join(data_root_path, "image", person_item)

cloth_item = "00006.jpg"  # 상의 또는 하의 이미지 파일명
cloth_type = "upper"  # 상의 또는 하의
cloth_path = os.path.join(data_root_path, "cloth", f"{cloth_type}_img", cloth_item)

cloth_type = "upper"  # 상의 또는 "lower"로 변경 가능

assert os.path.exists(person_path), f"Person image not found at {person_path}"
assert os.path.exists(cloth_path), f"Cloth image not found at {cloth_path}"

# 이미지 로드 및 전처리
person_img = Image.open(person_path).convert("RGB")
cloth_img = Image.open(cloth_path).convert("RGB")

# 무관성 마스크 생성
mask = automasker(person_path, mask_type=cloth_type)['mask']

# 텐서로 변환
person_tensor = prepare_image(person_img).to(device)
cloth_tensor = prepare_image(cloth_img).to(device)
mask_tensor = prepare_mask_image(mask).to(device)

# 추론 설정
num_inference_steps = 50
guidance_scale = 2.5
generator = torch.Generator(device=device).manual_seed(-1)

# LoRA 적용된 파이프라인으로 추론
pipeline.unet.eval()  # 추론 모드로 전환
with torch.no_grad():
    result = pipeline(
        image=person_tensor,
        condition_image=cloth_tensor,
        mask=mask_tensor,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        height=1024, 
        width=768,   
        generator=generator,
    )

# 이미지 저장
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
result_save_path = os.path.join(output_dir, f"lora_result_{person_item}_{cloth_item}_{cloth_type}_{timestamp}.jpg")
result[0].save(result_save_path)
print(f"Result image saved to {result_save_path}")

Using device: cuda


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Model downloaded to: C:\Users\coldbrew\.cache\huggingface\hub\models--zhengchong--CatVTON\snapshots\2969fcf85fe62f2036605716f0b56f0b81d01d79


c:\Users\coldbrew\VTON-project\CatVTON\model\SCHP\__init__.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(ckpt_path, map_location='cpu')['state

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Downloaded zhengchong/CatVTON to C:\Users\coldbrew\.cache\huggingface\hub\models--zhengchong--CatVTON\snapshots\2969fcf85fe62f2036605716f0b56f0b81d01d79
Downloaded LoRA weights from Coldbrew9/wheel-CatVTON to infer\first_best_loss_lora_r4_model_14.pt
Extracted lora_rank from filename: 4
Loaded LoRA weights from infer\first_best_loss_lora_r4_model_14.pt


C:\Users\coldbrew\AppData\Local\Temp\ipykernel_3480\782156775.py:83: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lora_state_dict = torch.load(lora_weights_path, map_locati

Result image saved to infer\lora_result_00003.jpg_00006.jpg_upper.jpg
